# Plant Leaf Disease Classifier

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#import os
#os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from keras.optimizers import SGD, RMSprop

In [2]:
import os
import pandas as pd
import numpy as np


from keras.preprocessing.image import ImageDataGenerator

In [3]:
root = 'leaf_classif'
print(os.path.join(root, 'train'))

leaf_classif\train


In [4]:
files = [file for file in os.listdir(root) if os.path.isfile(os.path.join(root, file))]
print(files)

[]


# We extracted the pictures of leaves and gave them their corresponding labels

## All of the leaf pictures have the dimensions 6000 x 4000 and 96 dpi for both horizontal and vertical resolution

In [5]:
import random

main_dir = root  # Replace with your main directory

def label_from_folder_name(folder_name):
    if 'healthy' in folder_name:
        return 'healthy'  # Class label for healthy images
    elif 'diseased' in folder_name:
        return 'diseased'  # Class label for unhealthy images
    else:
        return None  # No specific label found
    
def custom_flow_from_directory(directory, target_size, batch_size):
    filenames = []
    labels = []
    i = 0

    for folder in os.listdir(directory):
        # first 2 folders
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path):
                 for file in files:
                     filenames.append(os.path.join(root, file))
                     labels.append(label_from_folder_name(root))

    filenames = np.array(filenames)
    labels = np.array(labels, dtype=str)  # Ensure labels are strings

    return ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        pd.DataFrame({"filename": filenames, "class": labels}),
        x_col="filename",
        y_col="class",
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary'
    )

# Use the custom function to load data
batch_size = 32
img_height, img_width = 150, 150

train_dir = os.path.join(main_dir, 'train')
test_dir = os.path.join(main_dir, 'test')
valid_dir = os.path.join(main_dir, 'valid')

train_generator = custom_flow_from_directory(train_dir, (img_height, img_width), batch_size)
test_generator = custom_flow_from_directory(test_dir, (img_height, img_width), batch_size)
valid_generator = custom_flow_from_directory(valid_dir, (img_height, img_width), batch_size)


Found 4274 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.
Found 110 validated image filenames belonging to 2 classes.


# Originally, we tried to run the model on all of the leaves, but found that we do not have the necessary hardware resources to do this.
- We tried to use Google Colab, but the service began to limit our usage of GPU and we encountered difficulty in trying to fully run the program.

# So, we decided to run the model on one type of leaf - Alstonia Scholaris.

In [6]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# We used a Convolutional Neural Network (CNN) from the Keras library to build a sequential model.

# Convolutional layers
- The first convolutional layer (Conv2D) has 32 filters, each with a 3x3 kernel. It applies the Rectified Linear Unit (ReLU) activation function.
- The second convolutional layer has 64 filters with a 3x3 kernel and ReLU activation.
- The third convolutional layer has 128 filters with a 3x3 kernel and ReLU activation.
- After each convolutional layer, a max-pooling layer (MaxPooling2D) with a 2x2 pool size is applied. This reduces the spatial dimensions of the representation and captures the most important information.

# Dense layers

- The first fully connected layer (Dense) has 128 neurons with ReLU activation. It processes the flattened features from the previous layer.
- The second and final fully connected layer has 1 neuron with a sigmoid activation function. This neuron outputs the probability of belonging to the positive class in binary classification tasks.

# Model Compilation

- The model is compiled using the Adam optimizer ('adam'), a popular optimization algorithm.
- The loss function used for training is binary crossentropy ('binary_crossentropy'), suitable for binary classification problems.
- The metric to monitor during training is accuracy ('accuracy').


In [7]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
steps_per_epoch_test = test_generator.samples // batch_size

epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
134/134 [==============================] - 13026s 98s/step - loss: 0.6324 - accuracy: 0.6282 - val_loss: 0.6097 - val_accuracy: 0.6182
Epoch 2/10
134/134 [==============================] - 774s 6s/step - loss: 0.4991 - accuracy: 0.7569 - val_loss: 0.6354 - val_accuracy: 0.6909
Epoch 3/10
134/134 [==============================] - 742s 6s/step - loss: 0.3726 - accuracy: 0.8379 - val_loss: 0.4356 - val_accuracy: 0.7818
Epoch 4/10
134/134 [==============================] - 763s 6s/step - loss: 0.2597 - accuracy: 0.8924 - val_loss: 0.6547 - val_accuracy: 0.8091
Epoch 5/10
134/134 [==============================] - 799s 6s/step - loss: 0.1885 - accuracy: 0.9242 - val_loss: 0.3928 - val_accuracy: 0.8273
Epoch 6/10
134/134 [==============================] - 786s 6s/step - loss: 0.1707 - accuracy: 0.9321 - val_loss: 0.3341 - val_accuracy: 0.9000
Epoch 7/10
134/134 [==============================] - 5203s 39s/step - loss: 0.1102 - accuracy: 0.9572 - val_loss: 0.3415 - val_accuracy: 0

In [8]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

4/4 [==============================] - 52s 12s/step - loss: 0.5944 - accuracy: 0.8545


# Fine tune CNN model - doubling number of epochs to 20, reducing steps size by half

In [9]:
# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
steps_per_epoch_test = test_generator.samples // (batch_size * 2)

epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1
if test_generator.samples % batch_size != 0:
    steps_per_epoch_test += 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
67/67 [==============================] - 1067s 16s/step - loss: 0.0296 - accuracy: 0.9907 - val_loss: 0.3337 - val_accuracy: 0.8594
Epoch 2/20
67/67 [==============================] - 1077s 16s/step - loss: 0.0295 - accuracy: 0.9907 - val_loss: 0.6129 - val_accuracy: 0.8438
Epoch 3/20
67/67 [==============================] - 1050s 16s/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.7386 - val_accuracy: 0.8906
Epoch 4/20
67/67 [==============================] - 438s 7s/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.6080 - val_accuracy: 0.8750
Epoch 5/20
67/67 [==============================] - 434s 6s/step - loss: 0.0299 - accuracy: 0.9901 - val_loss: 0.8184 - val_accuracy: 0.8438
Epoch 6/20
67/67 [==============================] - 417s 6s/step - loss: 0.0241 - accuracy: 0.9920 - val_loss: 0.6099 - val_accuracy: 0.8750
Epoch 7/20
67/67 [==============================] - 411s 6s/step - loss: 0.0166 - accuracy: 0.9939 - val_loss: 0.4986 - val_accuracy: 0.9062
Epoch 8

# This fine-tuned model had slightly worse accuracy than the previous model.

# Perhaps this can be attributed to over-fitting introduced by further tuning.

In [10]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=steps_per_epoch_test)

2/2 [==============================] - 27s 14s/step - loss: 0.7436 - accuracy: 0.8906


Use a model built from a pretrained model

In [11]:
# Use pretrained model
from keras.applications import MobileNet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

# Define input image size expected by MobileNet
img_width, img_height = 224, 224

# Load the MobileNet model without the top classification layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification using sigmoid activation

# Combine base model with custom top layers
mobileModel = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
mobileModel.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_valid = valid_generator.samples // batch_size
epochs = 10
# Add 1 extra step if there are remaining samples not included in batches
if train_generator.samples % batch_size != 0:
    steps_per_epoch_train += 1
if valid_generator.samples % batch_size != 0:
    steps_per_epoch_valid += 1

mobileModel.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/10
134/134 [==============================] - 1822s 14s/step - loss: 0.4102 - accuracy: 0.8119 - val_loss: 0.5509 - val_accuracy: 0.7727
Epoch 2/10
134/134 [==============================] - 1878s 14s/step - loss: 0.2653 - accuracy: 0.8830 - val_loss: 0.4834 - val_accuracy: 0.7727
Epoch 3/10
134/134 [==============================] - 1911s 14s/step - loss: 0.2275 - accuracy: 0.9003 - val_loss: 0.5646 - val_accuracy: 0.7636
Epoch 4/10
134/134 [==============================] - 1925s 14s/step - loss: 0.1840 - accuracy: 0.9249 - val_loss: 0.4640 - val_accuracy: 0.8091
Epoch 5/10
134/134 [==============================] - 1259s 9s/step - loss: 0.1616 - accuracy: 0.9378 - val_loss: 0.4305 - val_accuracy: 0.8182
Epoch 6/10
134/134 [==============================] - 744s 6s/step - loss: 0.1423 - accuracy: 0.9420 - val_loss: 0.4693 - val_accuracy: 0.8455
Epoch 7/10
134/134 [==============================] - 855s 6s/step - loss: 0.1129 - accuracy: 0.9569 - val_loss: 0.4209 - val_accurac

# The pretrained model performed poorly. The accuracy was about chance-level.
- The pretrained model did not seem to fit our specific classification problem well.

In [13]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

4/4 [==============================] - 21s 5s/step - loss: 0.3049 - accuracy: 0.8909
Test Loss: 0.3049
Test Accuracy: 89.09%


In [14]:
# Fine tune the pretrained model

# Calculate steps_per_epoch and validation_steps
steps_per_epoch_train = train_generator.samples // (batch_size * 2)
steps_per_epoch_valid = valid_generator.samples // (batch_size * 2)
epochs = 20
# Add 1 extra step if there are remaining samples not included in batches
history = mobileModel.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_valid
)

Epoch 1/20
66/66 [==============================] - 434s 7s/step - loss: 0.0426 - accuracy: 0.9914 - val_loss: 0.5282 - val_accuracy: 0.8438
Epoch 2/20
66/66 [==============================] - 421s 6s/step - loss: 0.0358 - accuracy: 0.9952 - val_loss: 0.9526 - val_accuracy: 0.6875
Epoch 3/20
66/66 [==============================] - 424s 6s/step - loss: 0.0368 - accuracy: 0.9938 - val_loss: 0.6260 - val_accuracy: 0.8438
Epoch 4/20
66/66 [==============================] - 432s 7s/step - loss: 0.0320 - accuracy: 0.9924 - val_loss: 0.5312 - val_accuracy: 0.7812
Epoch 5/20
66/66 [==============================] - 428s 6s/step - loss: 0.0260 - accuracy: 0.9972 - val_loss: 0.4978 - val_accuracy: 0.8750
Epoch 6/20
66/66 [==============================] - 431s 7s/step - loss: 0.0269 - accuracy: 0.9953 - val_loss: 0.2949 - val_accuracy: 0.8438
Epoch 7/20
66/66 [==============================] - 426s 6s/step - loss: 0.0218 - accuracy: 0.9967 - val_loss: 0.3976 - val_accuracy: 0.9062
Epoch 8/20
66

# Fine-tuning the model did not improve it.

In [15]:
# Evaluate the model on the test dataset
loss, accuracy = mobileModel.evaluate(test_generator)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

4/4 [==============================] - 20s 5s/step - loss: 0.4559 - accuracy: 0.9000
Test Loss: 0.4559
Test Accuracy: 90.00%


# Conclusion

## Our best model had an accuracy of 83.64% on the test set

## This best model is overfitting the training dataset. Most of the error in our model can likely be attributed to overfitting the training dataset, as it reaches 100% accuracy in the training sets quickly.

## Another factor that could be causing non-optimal accuracy is the low image resolution, since the original images were too large and we had to scale them down in the code